<a href="https://colab.research.google.com/github/AnomalousBadge5/INF974_P2/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.2.1+cu121


In [3]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


Looking in links: https://pytorch-geometric.com/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 49.7 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 74.0 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 47.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm

In [5]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6))

data=dataset[0]

Extracting /tmp/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!


In [6]:
data

Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1], edge_index=[2, 15108])

In [7]:
num_classes = len(torch.unique(data.y))
print("Number of classes:", num_classes)

Number of classes: 4


In [8]:
data.x

tensor([[-0.0392,  0.3344,  0.9416],
        [ 0.0011,  0.3488, -0.9372],
        [-0.2507,  0.9366,  0.2447],
        ...,
        [ 0.6270, -0.5863,  0.5130],
        [-0.2090,  0.9760, -0.0607],
        [-0.2459,  0.9653, -0.0878]])

In [9]:
dataset[0]

Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1], edge_index=[2, 15108])

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [22]:
dataset

ShapeNet(2349, categories=['Airplane'])

In [7]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: ShapeNet(2349, categories=['Airplane']):
Number of graphs: 2349
Number of features: 3
Number of classes: 50
Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1], edge_index=[2, 15108])
Number of nodes: 2518
Number of edges: 15108
Average node degree: 6.00
Has isolated nodes: False
Has self-loops: False
Is undirected: False


In [9]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:1879]
test_dataset = dataset[1879:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 1879
Number of test graphs: 470


In [10]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[164933, 3], y=[164933], pos=[164933, 3], category=[64], edge_index=[2, 989598], batch=[164933], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[163853, 3], y=[163853], pos=[163853, 3], category=[64], edge_index=[2, 983118], batch=[163853], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[165033, 3], y=[165033], pos=[165033, 3], category=[64], edge_index=[2, 990198], batch=[165033], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[164750, 3], y=[164750], pos=[164750, 3], category=[64], edge_index=[2, 988500], batch=[164750], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[163703, 3], y=[163703], pos=[163703, 3], category=[64], edge_index=[2, 982218], batch=[163703], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[164545, 3], y=[164545], pos=[164545, 3], category=[64], edge_index=[2, 987270], batch=[

In [34]:
data.y.unique()

tensor([0, 1, 2, 3])

In [26]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

<IPython.core.display.Javascript object>

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.lin = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.lin(x)
        return x


In [52]:
model = GNNModel(3, 64, 4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [53]:
model

GNNModel(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=4, bias=True)
)

In [54]:
model=  model.to(device)

In [57]:
def train(model, optimizer, criterion, train_loader):
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index)
            pred = out.argmax(dim=1)
            correct += (pred == data.y).sum().item()
            total += data.y.size(0)
    return correct / total

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    train(model, optimizer, criterion, train_loader)
    test_acc = test(model, test_loader)
    print(f'Epoch {epoch + 1}, Test Accuracy: {test_acc:.4f}')

Epoch 1, Test Accuracy: 0.7051
Epoch 2, Test Accuracy: 0.7149
Epoch 3, Test Accuracy: 0.7248
Epoch 4, Test Accuracy: 0.7301
Epoch 5, Test Accuracy: 0.7331
Epoch 6, Test Accuracy: 0.7274
Epoch 7, Test Accuracy: 0.7366
Epoch 8, Test Accuracy: 0.7402
Epoch 9, Test Accuracy: 0.7380
Epoch 10, Test Accuracy: 0.7412
